![New Release: Accelerate YOLOv8](assets/yolov8.png)

# Accelerate Ultralytics YOLOv8 with Speedster

In [21]:
# model_name: str = "yolov8n-seg.pt"
model_name: str = 'yolov8s.pt'

Hi and welcome 👋

In this notebook we will discover how in just a few steps you can speed up the response time of deep learning model inference using the Speedster module from the open-source library nebullvm.

With Speedster's latest API, you can speed up models up to 10 times without any loss of accuracy (option A), or accelerate them up to 20-30 times by setting a self-defined amount of accuracy/precision that you are willing to trade off to get even lower response time (option B). To accelerate your model, Speedster takes advantage of various optimization techniques such as deep learning compilers (in both option A and option B), quantization, half accuracy, and so on (option B).

Let's jump to the code.

In [22]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


## Setup

### Install Speedster

In [23]:
!pip install speedster

In [24]:
!python -m nebullvm.installers.auto_installer --frameworks torch --compilers all

2023-03-15 20:35:38 | INFO     | Running auto install of nebullvm dependencies
--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/segme

### Install Ultralytics YOLOv8

In [25]:
!pip install ultralytics


## Load YOLOv8s

In [26]:
import torch
from ultralytics import YOLO

yolo = YOLO(model_name)

Let's load a test dummy data and see the original output

In [27]:
test_data = torch.randn(1, 3, 640, 640)
yolo.model(test_data) # type: ignore

(tensor([[[4.14859e+00, 1.48655e+01, 2.25781e+01,  ..., 5.34127e+02, 5.46429e+02, 5.62946e+02],
          [1.08360e+01, 5.99086e+00, 6.39432e+00,  ..., 6.15986e+02, 6.09265e+02, 6.04129e+02],
          [8.22145e+00, 2.57848e+01, 3.48391e+01,  ..., 2.06990e+02, 2.00341e+02, 1.93398e+02],
          ...,
          [5.63576e-07, 4.81692e-07, 3.54388e-07,  ..., 1.99650e-06, 1.51591e-06, 9.15296e-07],
          [4.00944e-07, 3.38612e-07, 2.76382e-07,  ..., 2.28068e-06, 2.05438e-06, 1.43168e-06],
          [5.77944e-07, 2.56105e-07, 1.61813e-07,  ..., 1.94415e-06, 1.64060e-06, 1.15275e-06]]]),
 [tensor([[[[  8.81701,   5.71406,   4.18604,  ...,   6.16044,   8.18270,   7.29705],
            [  9.33525,   5.62313,   3.90346,  ...,   4.38414,   7.75076,   6.59148],
            [  9.29873,   4.92093,   3.18499,  ...,   2.53035,   7.02269,   5.97100],
            ...,
            [  9.74313,   5.21258,   2.78604,  ...,   3.17034,   5.78247,   8.97089],
            [  9.03045,   4.89194,   2.90346,

The original YOLOv8 model return as output a tuple where the first element is a tensor and the second is a list of tensors. Speedster currently supports only models that return only tensors, so we need to create a wrapper to overcome this issue:

In [28]:
class YOLOWrapper(torch.nn.Module):
    def __init__(self, yolo_model):
        super().__init__()
        self.model = yolo_model.model
    
    def forward(self, x, *args, **kwargs):
        res = self.model(x)
        return res[0], *tuple(res[1])
        
model_wrapper = YOLOWrapper(yolo)

## YOLOv8s Optimization with GPU

We can now optimize the model using speedster:

In [29]:
from speedster import optimize_model

# Provide some input data for the model    
input_data = [((torch.randn(1, 3, 640, 640), ), torch.tensor([0])) for i in range(100)]

# Run Speedster optimization
optimized_model = optimize_model(
  model_wrapper, input_data=input_data, metric_drop_ths=0.1, store_latencies=True
)

2023-03-15 20:36:01 | INFO     | Running Speedster on CPU
2023-03-15 20:36:03 | WARNING  | Missing Frameworks: tensorflow.
 Please install them to include them in the optimization pipeline.
2023-03-15 20:36:03 | WARNING  | Missing Compilers: onnxruntime, deepsparse, neural_compressor.
 Please install them to include them in the optimization pipeline.
2023-03-15 20:36:57 | INFO     | Benchmark performance of original model


KeyboardInterrupt: 

We can finally restore the original output format by wrapping the optimized model in a new class:

In [30]:
class OptimizedYOLO(torch.nn.Module):
    def __init__(self, optimized_model):
        super().__init__()
        self.model = optimized_model
    
    def forward(self, x, *args, **kwargs):
        res = self.model(x)
        return res[0], list(res[1:])
    
optimized_wrapper = OptimizedYOLO(optimized_model)

In [ ]:
optimized_wrapper(test_data.cuda())

## YOLOv8s Optimization with CPU

In [31]:
from speedster import optimize_model, save_model, load_model
from ultralytics import YOLO

yolo = YOLO(model_name)
model_wrapper = YOLOWrapper(yolo)

# Provide some input data for the model    
input_data = [((torch.randn(1, 3, 640, 640), ), torch.tensor([0])) for i in range(100)]

# Run Speedster optimization
optimized_model = optimize_model(
  model_wrapper, input_data=input_data, metric_drop_ths=0.1, store_latencies=True, device="cpu"
)

optimized_wrapper = OptimizedYOLO(optimized_model)

2023-03-15 20:37:59 | INFO     | Running Speedster on CPU
2023-03-15 20:38:00 | WARNING  | Missing Frameworks: tensorflow.
 Please install them to include them in the optimization pipeline.
2023-03-15 20:38:00 | WARNING  | Missing Compilers: onnxruntime, deepsparse, neural_compressor.
 Please install them to include them in the optimization pipeline.


In [ ]:
optimized_wrapper(test_data)

## Save and reload the optimized model

We can easily save to disk the optimized model with the following line:

In [ ]:
save_model(optimized_model, "model_save_path")

We can then load again the model:

In [ ]:
optimized_model = load_model("model_save_path")
optimized_wrapper = OptimizedYOLO(optimized_model)

What an amazing result, right?!? Stay tuned for more cool content from the Nebuly team :) 

<center> 
    <a href="https://discord.com/invite/RbeQMu886J" target="_blank" style="text-decoration: none;"> Join the community </a> |
    <a href="https://nebuly.gitbook.io/nebuly/welcome/questions-and-contributions" target="_blank" style="text-decoration: none;"> Contribute to the library </a>
</center>

<center> 
    <a href="https://github.com/nebuly-ai/nebullvm/tree/main/apps/accelerate/speedster#key-concepts" target="_blank" style="text-decoration: none;"> How speedster works </a> •
    <a href="https://github.com/nebuly-ai/nebullvm/tree/main/apps/accelerate/speedster#documentation" target="_blank" style="text-decoration: none;"> Documentation </a> •
    <a href="https://github.com/nebuly-ai/nebullvm/tree/main/apps/accelerate/speedster#quick-start" target="_blank" style="text-decoration: none;"> Quick start </a> 
</center>